# Results analysis

In [31]:
import pandas as pd
from scipy import stats
import numpy as np
import requests

## Significance test between projects results

In [108]:
repos = pd.read_csv("repos.csv", )
results = []

# Get results
for url in repos["Git"]:
    url_master = url.replace("github.com", "raw.githubusercontent.com") + "/refs/heads/master/answer_associativity.txt"
    response = requests.get(url_master)
    if response.status_code == 200:
        results.append(response.text.strip())
    else:
        url_main = url.replace("github.com", "raw.githubusercontent.com") + "/refs/heads/main/answer_associativity.txt"
        response = requests.get(url_main)
        if response.status_code == 200:
            results.append(response.text.strip())
        else:
            print("Foiré noob", url_master)
            results.append("")

repos["results"] = results

Foiré noob https://raw.githubusercontent.com/l55I1/REP/refs/heads/master/answer_associativity.txt
Foiré noob https://raw.githubusercontent.com/Humilokaki/TP2-REP-INSA-202425/refs/heads/master/answer_associativity.txt


In [109]:
repos

,Binome1,Binome2,Sujet,Git,NRep,results
0,Matys Couroussé,Axel Homery,Sujet 2,https://github.com/Matys53/REP_TP,1000000,93.11
1,Solène Lemonnier,Pauline Roches,Sujet 2,https://github.com/PaulineRoches/REP_popo_sos,50000,92.45
2,Chenyu Li,Bohan Ha,Sujet 3,https://github.com/l55I1/REP,1000000,
3,Victor van Kempen,Matthieu Hillairet,Sujet 1,https://github.com/matth1446/rep_mh_vvk,100,79.71
4,Antoine MARCHAL DOMBRAT,Jean Haurogné 🤓,Sujet 1,https://github.com/AntoineMarchalDombrat/REP_A...,100000,77.38
5,Ezgi Ozel,Alp Jakop,Sujet 2,https://github.com/alpjakop/REP,1000000,100.00
6,Noé Bourmalo,Aidan Gallagher,Sujet 2,https://github.com/Humilokaki/TP2-REP-INSA-202425,512,
7,Jeremy Brunet,Arno Lecrivain,Sujet 2,https://github.com/Mistyycs/REP-TP2,1000,82.79
8,Noa Berton,Guillaume Freyermuth,Sujet 1,https://github.com/Kaeios/assoc-REP/,100,100.00
9,Alexis Simon,Malvin Chevallier,Sujet 3,https://github.com/Dyskal/repro,125000,78.48


In [110]:
# Parse floats
repos = repos[pd.to_numeric(repos["results"], errors="coerce").notna()]
repos["results"] = repos["results"].astype("float") / 100.0
repos.reset_index(inplace=True, drop=True)
repos

/tmp/ipykernel_55855/465093398.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  repos["results"] = repos["results"].astype("float") / 100.0


,Binome1,Binome2,Sujet,Git,NRep,results
0,Matys Couroussé,Axel Homery,Sujet 2,https://github.com/Matys53/REP_TP,1000000,0.9311
1,Solène Lemonnier,Pauline Roches,Sujet 2,https://github.com/PaulineRoches/REP_popo_sos,50000,0.9245
2,Victor van Kempen,Matthieu Hillairet,Sujet 1,https://github.com/matth1446/rep_mh_vvk,100,0.7971
3,Antoine MARCHAL DOMBRAT,Jean Haurogné 🤓,Sujet 1,https://github.com/AntoineMarchalDombrat/REP_A...,100000,0.7738
4,Ezgi Ozel,Alp Jakop,Sujet 2,https://github.com/alpjakop/REP,1000000,1.0000
5,Jeremy Brunet,Arno Lecrivain,Sujet 2,https://github.com/Mistyycs/REP-TP2,1000,0.8279
6,Noa Berton,Guillaume Freyermuth,Sujet 1,https://github.com/Kaeios/assoc-REP/,100,1.0000
7,Alexis Simon,Malvin Chevallier,Sujet 3,https://github.com/Dyskal/repro,125000,0.7848


In [111]:
# Compute mean2mean stats
p_values = np.ones((len(repos), len(repos)))

for i in range(len(repos)):
    for j in range(i, len(repos)):
        sd1 = np.sqrt(repos["results"][i]*(1-repos["results"][i])) * ((repos["NRep"][i] + 1) / repos["NRep"][i]) if repos["results"][i] != 1.0 else 0.01
        sd2 = np.sqrt(repos["results"][j]*(1-repos["results"][j])) * ((repos["NRep"][i] + 1) / repos["NRep"][i]) if repos["results"][i] != 1.0 else 0.01
        t_stat, p_val = stats.ttest_ind_from_stats(
            mean1=repos["results"][i], std1=sd1, nobs1=repos["NRep"][i],
            mean2=repos["results"][j], std2=sd2, nobs2=repos["NRep"][j],
            equal_var=False
        )
        p_values[i, j] = p_val
        p_values[j, i] = p_val

df_p_values = pd.DataFrame(p_values)
df_p_values

,0,1,2,3,4,5,6,7
0,1.000000e+00,4.730670e-08,0.001213,0.000000e+00,0.000000,2.131476e-17,0.000000e+00,0.000000e+00
1,4.730670e-08,1.000000e+00,0.002050,0.000000e+00,0.000000,2.238067e-15,0.000000e+00,0.000000e+00
2,1.213384e-03,2.050450e-03,1.000000,5.677207e-01,0.000003,4.687138e-01,2.529918e-06,7.627565e-01
3,0.000000e+00,0.000000e+00,0.567721,1.000000e+00,0.000000,7.412625e-06,0.000000e+00,4.217282e-10
4,0.000000e+00,0.000000e+00,0.000003,0.000000e+00,1.000000,0.000000e+00,1.000000e+00,0.000000e+00
5,2.131476e-17,2.238067e-15,0.468714,7.412625e-06,0.000000,1.000000e+00,7.071111e-43,3.462329e-04
6,0.000000e+00,0.000000e+00,0.000003,0.000000e+00,1.000000,7.071111e-43,1.000000e+00,3.386154e-134
7,0.000000e+00,0.000000e+00,0.762756,4.217282e-10,0.000000,3.462329e-04,3.386154e-134,1.000000e+00


Significance test results between our project (n°5) and the others:

In [112]:
df_p_values.iloc[[5]]

,0,1,2,3,4,5,6,7
5,2.131476e-17,2.238067e-15,0.468714,0.000007,0.0,1.0,7.071111e-43,0.000346


### Interpretation:

Generally speaking, the main variability factors are:
- Numbers internal representation (floating point, symbolic computing...) and the corresponding algorithms used by the language
- Precision (ie number of bits: float, double...)
- Size of the numbers (floats are more precise near 0)
- Language, librairies and other dependancies versions
- Number of trials
- Coding errors...

Our result of 100% is due to the use of VPA (variable precision arithmetic), allowing octave to compute both sums symbolically. This explains its very low P-value (the closer to 1, the stronger the correlation is) with all the other projects, that uses floating point approximations.

According to the P-values, the only project with a similar result is n°3 (Java). However, it is due to an coding mistake: the number are generated randomly as floats, and then casted to doubles to perform the check. When the numbers are generated as doubles in the first place, we obtain an output of 82.82%.

## Variability factors exploration

We tried different values for each of the following parameters, in our Octave project:

### For 100 trials and the default predicate:

In [103]:
result1 = np.array([[0.82, 0.58],
                    [0.83, 0.71],
                    [0.84, 1],
                    [0.77, 1],
                    [0.79, 1]])
pd.DataFrame(result1, columns=["Float", "VPA"], index=["1 digit", "8 digits", "16 digits", "32 digits", "64 digits"])

,Float,VPA
1 digit,0.82,0.58
8 digits,0.83,0.71
16 digits,0.84,1.00
32 digits,0.77,1.00
64 digits,0.79,1.00


### For 100 trials, with different predicates:

In [104]:
result2 = np.array([[0.82, 1],
                    [0.85, 1]])
pd.DataFrame(result2, columns=["Float", "VPA"], index=["(x+y)+z == x+(y+z)", "((x+y)+z) - (x+(y+z)) == 0.0"])

,Float,VPA
(x+y)+z == x+(y+z),0.82,1.0
((x+y)+z) - (x+(y+z)) == 0.0,0.85,1.0


### For 10 000 trials, with different predicates:

In [105]:
result3 = np.array([[0.8271],
                    [0.8195]])
pd.DataFrame(result3, columns=["Float"], index=["(x+y)+z == x+(y+z)", "((x+y)+z) - (x+(y+z)) == 0.0"])

,Float
(x+y)+z == x+(y+z),0.8271
((x+y)+z) - (x+(y+z)) == 0.0,0.8195
